# Angle Optimization
## Optimizing combination of t1 and t3
Abby Bechtel

In [14]:
import interferometer as ir
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Set default plot line color cycling
plt.rcParams.update(plt.rcParamsDefault)
ccycle = mpl.cycler('color', plt.cm.rainbow(np.linspace(0.1, 2.9, 30)))
plt.rc('axes', prop_cycle = ccycle)

# Allow fancy axis number formatting
from matplotlib.ticker import EngFormatter, ScalarFormatter

# Set stylesheet for plots
plt.style.use("seaborn-pastel")
mpl.style.use('bmh')

## Taking gradient

In [5]:
# set angles for testing 
t1 = 0
t3 = 0
t2min = 0
t2max = 180

In [6]:
t2s, h = ir.intensity(t1,t3,0,t2min,t2max) # domain and horizontal
_  , v = ir.intensity(t1,t3,90,t2min,t2max) # vertical

# Split-detected signal
s = (h - v)/(h + v)

<ipython-input-6-88c15284c6f4>:5: RuntimeWarning: invalid value encountered in true_divide
  s = (h - v)/(h + v)


In [7]:
def gradient(x):
    n=len(x)
    d = np.eye(n,n,1)-np.eye(n,n,-1)
    d[0][0:2]=np.array([[-2,2]])
    d[n-1,n-2:n]=np.array([[-2,2]])
    D=d/(2*(max(x)-min(x))/(len(x)-1))
    return D

def get_points(d1,d2):
    l = []
    for i in range(len(d1)):
        if d1[i] != 0:
            if abs(d1[i] - d2[i]) <= 0.000001:
                l.append(d1[i])
    return l

def graph(fx, xpoints):
    yDeriv1 = gradient(xpoints)@fx
    yDeriv2 = gradient(xpoints)@yDeriv1
    yFunc = fx
    p = get_points(yDeriv1,yDeriv2)
    
    f = plt.figure(figsize=(8,8))
    a = plt.axes()
    # Explicitly plot matched pairs of x-y points on the axis a
    a.plot(xpoints, yFunc, label="$f(x)$")
    # Explicitly plot different matched pair of x-y points in a different color
    a.plot(xpoints, yDeriv1, color="Red", label="$f^ {'}(x)$")
    # Explicitly plot different matched pair of x-y points in a different color
    a.plot(xpoints, yDeriv2, color="Green", label="$f^ {''}(x)$")
    # Explicitly set axis labels and a plot title
    a.set(xlabel="Domain", ylabel="Range", title="Derivative Approximation")
    # Add a legend describing which curve is which
    a.legend()
    # Show the active plot to the screen
    plt.show()
    return yDeriv1, yDeriv2

In [8]:
# return max slope for all combinations of polarizer 1 and 2 angles within x array
def test_slopes(x):
    slope_map = {}
    
    # i is first polarizer - t1
    for i in x:
        # j is third polarizer - t3
        for j in x:

            # Dark-port polarization intensities
            t2s, h = ir.intensity(i,j,0,0,180)
            _  , v = ir.intensity(i,j,90,0,180)

            # Split-detected signal
            s = (h - v)/(h + v)
        
            # gradient array over domain
            grad_arr = np.abs(gradient(s)@t2s)
            # returning either max 5 slopes for i,j polarization combination or simply 1 max slope
#             ind_max5 = np.argpartition(grad_arr, -5)[-5:]
            ind_max = np.argpartition(grad_arr, -1)[-1:]
            # creating dictionary with (i,j) key and value of the max slope value and the index
            slope_map[(i,j)]=(grad_arr[ind_max],t2s[ind_max])
    return slope_map

In [15]:
domain = np.linspace(0,180,181)
slope_dict = test_slopes(domain)

<ipython-input-8-3ce5cf0c66d4>:15: RuntimeWarning: invalid value encountered in true_divide
  s = (h - v)/(h + v)
<ipython-input-7-eaf7c4809104>:6: RuntimeWarning: divide by zero encountered in true_divide
  D=d/(2*(max(x)-min(x))/(len(x)-1))
<ipython-input-7-eaf7c4809104>:6: RuntimeWarning: invalid value encountered in true_divide
  D=d/(2*(max(x)-min(x))/(len(x)-1))
<ipython-input-8-3ce5cf0c66d4>:18: RuntimeWarning: invalid value encountered in matmul
  grad_arr = np.abs(gradient(s)@t2s)


In [16]:
# get 5 max slopes from all combinations in max_slope dictionary
keys, values = zip(*slope_dict.items())

# get slopes from tuple containing slopes and indexes for slopes
slopes = np.array([tuple[0][0] for tuple in values])
slopes  = slopes[~np.isnan(slopes)]

# get max 5 slopes
ind_max = np.argpartition(slopes, -5)[-5:]
# max 5 slopes for all combinations of i,j
max_slopes = slopes[ind_max]

In [17]:
p1p3_combination = []
p1p3_angles = []
# get the combinations of polarizers corrisponding to max slopes
for i in keys:
    for j in max_slopes:
        if slope_dict[i][0][0] == j:
            t1,t3 = i
            p1p3_combination.append([t1,t3,j])

In [18]:
df = pd.DataFrame(p1p3_combination, columns=['t1', 't3', 'max_slope'])

In [19]:
df = df.drop_duplicates()

In [20]:
df.sort_values(by=['max_slope'], ascending=False).reset_index(drop=True)

,t1,t3,max_slope
0,23.0,138.0,540049.436454
1,113.0,138.0,540049.436454
2,138.0,23.0,540049.436454
3,138.0,113.0,540049.436454
4,23.0,48.0,540049.436454
5,42.0,67.0,540049.436454
6,42.0,157.0,540049.436454
7,48.0,23.0,540049.436454
8,48.0,113.0,540049.436454
9,67.0,42.0,540049.436454
